In [15]:
%%bash
# pip install datasketch[redis]
# pip install elasticsearch==7.10.0
# pip install neo4j
# pip install pyyaml
# pip install deepdiff

In [32]:
%%local

from livy import LivySession
from textwrap import dedent

LIVY_URL = 'http://localhost:8998'

with LivySession.create(LIVY_URL) as session:
    session.run('print("foo")')
   
    session.upload('files_df', files_df)

    session.run(dedent("""
        print(files_df)
    """))
    # top_ten_pandas.append(session.read('df'))
            file = filename.first().getString(0)
            df = spark.read.csv('/sample_data/')
            top_ten = df.limit(10)
top_ten_pandas

HTTPError: 500 Server Error: Server Error for url: http://localhost:8998/sessions/36/statements

In [23]:
%%local

# import parser object from tike
from tika import parser  

import tika
  
url = 'https://aclanthology.org/2020.acl-main.577.pdf'
url = 'http://127.0.0.1:9864/webhdfs/v1/grades.csv?op=OPEN&namenoderpcaddress=master:9000&offset=0'
pdfFile = parser.from_file(url)

print(pdfFile['content'])

print ("_mapping response:", json.dumps(pdfFile['metadata'], indent=4), "\n")













	Last name	First name	SSN	Test1	Test2	Test3	Test4	Final	Grade
	Alfalfa	Aloysius	123-45-6789	40	90	100	83	49	D-
	Alfred	University	123-12-1234	41	97	96	97	48	D+
	Gerty	Gramma	567-89-0123	41	80	60	40	44	C
	Android	Electric	087-65-4321	42	23	36	45	47	B-
	Bumpkin	Fred	456-78-9012	43	78	88	77	45	A-
	Rubble	Betty	234-56-7890	44	90	80	90	46	C-
	Noshow	Cecil	345-67-8901	45	11	-1	4	43	F
	Buff	Bif	632-79-9939	46	20	30	40	50	B+
	Airpump	Andrew	223-45-6789	49	1	90	100	83	A
	Backus	Jim	143-12-1234	48	1	97	96	97	A+
	Carnivore	Art	565-89-0123	44	1	80	60	40	D+
	Dandy	Jim	087-75-4321	47	1	23	36	45	C+
	Elephant	Ima	456-71-9012	45	1	78	88	77	B-
	Franklin	Benny	234-56-2890	50	1	90	80	90	B-
	George	Boy	345-67-3901	40	1	11	-1	4	B
	Heffalump	Harvey	632-79-9439	30	1	20	30	40	C


_mapping response: {
    "X-TIKA:Parsed-By": [
        "org.apache.tika.parser.DefaultParser",
        "org.apache.tika.parser.csv.TextAndCSVParser"
    ],
    "X-TIKA:Parsed-By-Full-Set": [
        "org.apache.tika.parser.

In [3]:
%%spark -o files_df

import ntpath
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, DecimalType

hadoop = sc._jvm.org.apache.hadoop

fs = hadoop.fs.FileSystem
conf = hadoop.conf.Configuration() 
path = hadoop.fs.Path('/sample_data/')

files = []
for f in fs.get(conf).listStatus(path):
    filename = ntpath.basename(str(f.getPath()))
    print(filename)
    files.append((filename, f.getLen()))


schema = StructType([
    StructField('Filename', StringType(), True),
    StructField('Length', StringType(), True)
])

# Convert list to RDD
rdd = spark.sparkContext.parallelize(files)

# Create data frame
files_df = spark.createDataFrame(rdd,schema)
files_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

adventureworks-for-postgres
+--------------------+------+
|            Filename|Length|
+--------------------+------+
|adventureworks-fo...|     0|
+--------------------+------+

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
%%local

from datetime import datetime
from datasketch import MinHash
from datasketch import MinHashLSH
from datasketch import LeanMinHash
from elasticsearch import Elasticsearch
from redis import StrictRedis
import redis
import json
import pickle
import base64
import numpy as np
import pandas as pd
import logging
import yaml
from yaml.loader import SafeLoader
from dataclasses import dataclass
from deepdiff import DeepDiff
from pprint import pprint
from typing import Dict, Any
import hashlib

logging.basicConfig(format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

config = []
config_data = []
try:
    with open('havel-config.yaml', 'r') as f:
        config = yaml.load(f, Loader=SafeLoader)
        for source in config['sources']:
            for schema in config['sources'][source]:
                url = config['sources'][source][schema]['url']
                datasets = config['sources'][source][schema]['datasets']
                for dataset in datasets:
                    dataframe = 0
                    config_data.append((source, schema, dataset, url, dataframe))

except yaml.YAMLError:
    print("Error in configuration file")

config_df = pd.DataFrame (config_data, columns = ['source', 'schema', 'dataset', 'url', 'dataframe'])
date = '2022-11-20'


In [2]:
%%send_to_spark -i config_df -t df -n data

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
46,application_1669190981900_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'config_df' as 'data' to Spark kernel

In [3]:
%%send_to_spark -i date -t str -n date

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'date' as 'date' to Spark kernel

In [4]:
%%spark

def read_data(i, date):
    abs_path = 'sample_data'
    source = data.rdd.map(lambda r: r.source).collect()[i]
    schema = data.rdd.map(lambda r: r.schema).collect()[i]
    dataset = data.rdd.map(lambda r: r.dataset).collect()[i]
    globals()[f"df_{i}"] = spark.read.format("csv").option("header", "true").load(f"/{abs_path}/{source}/{schema}/{date}/{dataset}.csv")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
%%spark -o df_0

read_data(0, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
%%spark -o df_1

read_data(1, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
%%spark -o df_2

read_data(2, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
%%spark -o df_3

read_data(3, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
%%spark -o df_4

read_data(4, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
%%spark -o df_5

read_data(5, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
%%spark -o df_6

read_data(6, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
%%spark -o df_7

read_data(7, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
%%spark -o df_8

read_data(8, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
%%spark -o df_9

read_data(9, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%spark -o df_10

read_data(10, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
%%spark -o df_11

read_data(11, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
%%spark -o df_12

read_data(12, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
%%spark -o df_13

read_data(13, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
%%spark -o df_14

read_data(14, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
%%spark -o df_15

read_data(15, date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
%%spark -o df_16

df_16 = spark.read.format("csv").option("header", "true").load("/sample_data/adventureworks-for-postgres/person/2022-11-23/address.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
%%local 
df_16

Output()

In [28]:
%%local 
df_0 = df_16

In [36]:
%%local

def add_df_to_data(data, df, i):
    row = list(data[i])
    row[4] = df
    data[i] = tuple(row)

for i in range(0, len(config_data)):
    add_df_to_data(config_data, globals()[f"df_{i}"], i)


In [37]:
%%local
config_data[0][4]

Output()

In [41]:
%%local


def calc_idx_content(data):
    return "%s-%s-%s-%s"%(data['source'].replace(' ', '_'), data['schema'].replace(' ', '_'), data['dataset'].replace(' ', '_'), data['attribute'].replace(' ', '_'))

def calc_idx_signature(data):
    return "%s-%s-%s"%(data['source'].replace(' ', '_'), data['schema'].replace(' ', '_'), data['dataset'].replace(' ', '_'))

def get_df_data_types_as_dict(df):
    return df.dtypes.apply(lambda x: x.name).to_dict()

def serialize_dict_data_types(data_types):
    return json.dumps(data_types, sort_keys=True)

def deserialize_dict_data_types(ser_df_data_types):
    return  json.loads(ser_df_data_types)

def compare_dicts(dict_a, dict_b):
    return DeepDiff(dict_a, dict_b)


def dict_hash(dictionary: Dict[str, Any]) -> str:
    """MD5 hash of a dictionary."""
    dhash = hashlib.md5()
    encoded = json.dumps(dictionary, sort_keys=True).encode()
    dhash.update(encoded)
    return dhash.hexdigest()


def calc_profile_content(source, schema, dataset, df):
    doc = []
    min_hashes = []
    
    for c_n in df.columns:
        m = MinHash(num_perm=128)
        s = df[c_n]
        dtype = str(s.dtype)
        crd = s.nunique()
        keywords = []
        
        for v in s:
            if dtype == 'int64': 
                v = str(v)
                m.update(v.encode('utf8'))
            elif dtype == 'datetime64[ns]': 
                v = v.strftime("%Y-%m-%d %H:%M:%S.%f")
                m.update(v.encode('utf8'))
            elif (isinstance(v, float)):
                 m.update((str(v)).encode('utf8'))
            else:
                m.update(v.encode('utf8'))
                keywords.append(v)
        
        data = {}
        data['source'] = source
        data['schema'] = schema
        data['dataset'] = dataset
        data['attribute'] = c_n
        data['data_type'] = dtype
        data['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
        data['size'] = s.size
        data['cardinality'] = crd
        data['uniqueness'] = crd / s.size     
        data['min_hash'] =  m.hashvalues
        data['min_hash_seed'] = m.seed
        data['keywords'] = ' '.join(keywords)
        
        min_hashes.append((calc_idx_content(data), m))
        doc.append(data)

    return doc, min_hashes


def calc_profile_signature(source, schema, dataset, df):
    doc = []
    min_hashes = []
    keywords = []
       
    m = MinHash(num_perm=128)
    for c in df.columns:
        m.update(c.encode('utf8'))
        keywords.append(c)
        
    data = {}
    data['source'] = source
    data['schema'] = schema
    data['dataset'] = dataset
    data['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    data['min_hash'] =  m.hashvalues
    data['min_hash_seed'] = m.seed
    data['data_types'] = serialize_dict_data_types(get_df_data_types_as_dict(df))
    data['keywords'] = ' '.join(keywords)
    
    min_hashes.append((calc_idx_signature(data), m))
    
    data['version'] = calc_version(calc_idx_signature(data), get_df_data_types_as_dict(df))
    
    doc.append(data)
    
    return doc, min_hashes



def calc_version(k, d):
    h = dict_hash(d)
    v = version_cache.get(k)
    version = 0
    
    print(v)
    if v:
        ser = json.loads(v)
        print(ser)
        print(k)
        version = int(ser['version'])
        if ser['hash'] != h:
            version = version + 1
            print('Version incremented for key: %s' % k)\
    
    nd = {
        'hash': h,
        'version': version
    }
    
    version_cache.set(k, json.dumps(nd))
        
    return version



def insert_min_hashes(min_hashes, lsh_list):
    for threshold, lsh in lsh_list.items():
        with lsh.insertion_session() as session:
            for key, minhash in min_hashes:
                label = key
                if debug: print(label)
                if debug: print(minhash)
                try:
                    lsh.remove(label)
                except:
                    print("Key %s doesn't exist."%label) 
                session.insert(label, minhash)


                
def upsert(env, index, func_calc_idx, doc, v=True):
    for data in doc:
        res = es.index(index=index, id=func_calc_idx(data), body=data)

    es.indices.refresh(index=index)
    
    if v is True:
        res = es.search(size=es_res_max_size, index=index, body={"query": {"match_all": {}}})
        if debug: print("Got %d Hits." % res['hits']['total']['value'])

        
def clear_cache_ns(ns):
    """
    Clears a namespace in redis cache.
    This may be very time consuming.
    :param ns: str, namespace i.e your:prefix*
    :return: int, num cleared keys
    """
    count = 0
    pipe = cache.pipeline()
    for key in cache.scan_iter(ns):
        pipe.delete(key)
        count += 1
    pipe.execute()
    return count


def clear_es(env):
    # delete all
    indices = [index_content[env], index_signature[env]]
        
    try:
        es.delete_by_query(index=indices, body={"query": {"match_all": {}}})
        es.indices.delete(index=indices, ignore=[400, 404])
    except Exception as e:
        print("Delete Index: %s", e)

def create_es_schema_signature(env):
    schema = {
        "mappings": {
            "properties": {
                "source": {
                    "type": "text" # formerly "string"
                },
                "schema": {
                    "type": "text" # formerly "string"
                },
                "dataset": {
                    "type": "text" # formerly "string"
                },
                "timestamp": {
                    "type": "date",
                    "format": "yyyy-MM-dd HH:mm:ss.SSSSSS"
                    # data format for Python's datetime.now() method
                },
                "min_hash": {
                    "ignore_malformed": "true",
                    "type": "integer"
                },
                "min_hash_seed": {
                    "type": "integer"
                },
                "data_types": {
                    "type": "text"
                },
                "version": {
                    "type": "integer"
                },
                "keywords": {
                    "type": "text"
                }
            }
        }
    }

    resp = es.indices.create(
        index = index_signature[env],
        body = schema
    )
    
    if debug: print ("_mapping response:", json.dumps(resp, indent=4), "\n")

    
def create_es_schema_content(env):
    schema = {
        "mappings": {
            "properties": {
                "source": {
                    "type": "text" # formerly "string"
                },
                "schema": {
                    "type": "text"
                },
                "dataset": {
                    "type": "text"
                },
                "attribute": {
                    "type": "text"
                },
                "data_type": {
                    "type": "text"
                },
                 "timestamp": {
                    "type": "date",
                    "format": "yyyy-MM-dd HH:mm:ss.SSSSSS"
                    # data format for Python's datetime.now() method
                },
                "size": {
                    "type": "integer"
                },
                "cardinality": {
                    "type": "float"
                },
                "uniqueness": {
                    "type": "float"
                },
                "min_hash": {
                    "ignore_malformed": "true",
                    "type": "integer"
                },
                "min_hash_seed": {
                    "type": "integer"
                },
                "keywords": {
                    "type": "text"
                }
            }
        }
    }

    resp = es.indices.create(
        index = index_content[env],
        body = schema
    )
    
    if debug: print ("_mapping response:", json.dumps(resp, indent=4), "\n")

    
def create_profiles_content(source, schema, dataset, df, env, lsh_list):
    doc, min_hashes = calc_profile_content(source, schema, dataset, df)
    insert_min_hashes(min_hashes, lsh_list)
    index = index_content[env]
    
    try:
        upsert(env, index, calc_idx_content, doc)
    except Exception as e:
        print("something didn't work", doc, "\n", str(e))
        

def create_profiles_signature(source, schema, dataset, df, env, lsh_list):
    doc, min_hashes = calc_profile_signature(source, schema, dataset, df)
    insert_min_hashes(min_hashes, lsh_list)
    index = index_signature[env]
    
    try:
        upsert(env, index, calc_idx_signature, doc)
    except Exception as e:
        print("something didn't work", doc, "\n", str(e))


def get_nodes_and_edges(env, index, lsh_list):
    res = es.search(size=es_res_max_size, index=index, body={"query": {"match_all": {}}})
    if debug: print("Got %d Hits." % res['hits']['total']['value'])

    relations = []

    for hit in res['hits']['hits']:
        min_hash_hashes = np.array(hit['_source']["min_hash"],  dtype=np.int64)
        min_hash_seed = hit['_source']["min_hash_seed"]

        mt = MinHash(num_perm=128)
        mt.hashvalues = min_hash_hashes
        mt.seed = min_hash_seed
        
        for threshold, lsh in lsh_list.items():
            bucket = lsh.query(mt)
            relations.append((hit['_id'], bucket, str(threshold)))
    return (res['hits']['hits'], relations)


def search_dataset_for_topic(index, topic):
    resp = es.search(
        index= index,
        body={
            "query": {
                "match": {
                    "keywords": topic
                }
            }
        }
    )
    
    res = []
    for r in resp['hits']['hits']:
        res.append({
            'id': r['_id'], 
            'score': r['_score'],
            'source': r['_source']['source'], 
            'schema': r['_source']['schema'], 
            'dataset': r['_source']['dataset'], 
            'attribute': r['_source']['attribute']})
    
    return res


In [42]:
%%local

from neo4j import GraphDatabase
from textwrap import dedent

class GraphStorage:

    def __init__(self, uri, user, password, env, debug):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.env = env
        self.debug = debug
        
    def close(self):
        self.driver.close()

    def insert_column_node(self, idx_col, idx_tab, data):
        with self.driver.session() as session:
            res1 = session.execute_write(self._create_column_node, idx_col, data, self.env)
            res2 = session.execute_write(self._create_column_table_relation, idx_col, idx_tab, self.env)
            if self.debug: print(res1)
            if self.debug: print(res2)
    
    def insert_table_node(self, idx, data):
        with self.driver.session() as session:
            res = session.execute_write(self._create_table_node, idx, data, self.env)
            if self.debug: print(res)
            
    def add_column_column_relation(self, idx_a, idx_b, threshold):
        with self.driver.session() as session:
            res = session.execute_write(self._create_column_column_relation, idx_a, idx_b, threshold, self.env)
            if self.debug: print(res)
    
    def add_table_table_relation(self, idx_a, idx_b, threshold):
        with self.driver.session() as session:
            res = session.execute_write(self._create_table_table_relation, idx_a, idx_b, threshold, self.env)
            if self.debug: print(res)
            
    def remove_self_relations(self):
        with self.driver.session() as session:
            res = session.execute_write(self._delete_self_relations, self.env)
            if self.debug: print(res)     
    
    def delete_all(self):
        with self.driver.session() as session:
            res = session.execute_write(self._delete_all, self.env)
            if self.debug: print(res)
    
    
    def find_related_tables_by_signature_to_2_grade(self, dataset, weight, dastart, daend):
        with self.driver.session() as session:
            res = session.execute_read(self._fetch_related_tables_by_signature_to_2_grade, dataset, weight, dastart, daend, self.env)
            if self.debug: print(res)

            return res
        
    def find_related_tables_by_content_to_1_grad(self, attribute, weight, dastart, daend):
        with self.driver.session() as session:
            res = session.execute_read(self._fetch_related_tables_by_content_to_1_grade, attribute, weight, dastart, daend, self.env)
            if self.debug: print(res)

            return res    
    
    
    @staticmethod
    def _create_column_node(tx, idx, data, env):
        result = tx.run("MERGE (a:Column {idx: $idx, source: $source, schema: $schema, dataset: $dataset, version: $version, env: $env, "
                        "attribute: $attribute, data_type: $data_type, uniqueness: $uniqueness}) "
                        "ON CREATE SET a.createdAt = timestamp(), a.updatetAt = timestamp() "
                        "ON MATCH SET a.updatetAt = timestamp() "
                        "RETURN a.name + ' created as node ' + id(a) ",
                        source=data['source'], schema=data['schema'], dataset=data['dataset'], version=1, timestamp=data['timestamp'],
                        attribute=data['attribute'],  data_type=data['data_type'], cardinality=data['cardinality'],  uniqueness=data['uniqueness'], 
                        env=env, idx=idx)
        
        return result.single()[0]
    
    
    @staticmethod
    def _create_table_node(tx, idx, data, env):
        result = tx.run("MERGE (a:Table{idx: $idx, source: $source, schema: $schema, dataset: $dataset, version: $version, env: $env}) "
                        "ON CREATE SET a.createdAt = timestamp(), a.updatetAt = timestamp() "
                        "ON MATCH SET a.updatetAt = timestamp() "
                        "RETURN a.name + ' created as node ' + id(a) ", 
                        source=data['source'], schema=data['schema'], dataset=data['dataset'], version=data['version'], timestamp=data['timestamp'], 
                        env=env, idx=idx)
        return result.single()[0]
    
    
    @staticmethod
    def _create_table_table_relation(tx, idx_a, idx_b, weight, env):
        result = tx.run("MATCH (a:Table {idx: $idx_a, env: $env}), (b:Table {idx: $idx_b, env: $env}) "
                        "MERGE (a)-[r:IS_SIMILAR_TO {weight: $weight}]-(b) "
                        "ON CREATE SET r.createdAt = timestamp() , r.updatetAt = timestamp()"
                        "ON MATCH SET r.updatetAt = timestamp() "
                        "RETURN 'relation ' + type(r) + ' created between ' + a.dataset + ' and ' + b.dataset",
                        idx_a=idx_a, idx_b=idx_b, weight=weight, env=env)
        
        entire_result = []
        for record in result:
            entire_result.append(record[0])
            
        return entire_result
    
    @staticmethod
    def _create_column_table_relation(tx, idx_col, idx_tab, env):
        result = tx.run("MATCH (t:Table {idx: $idx_tab, env: $env}) "
                        "WITH max(t.version) AS maximum "
                        "MATCH (c:Column {idx: $idx_col, env: $env}), (t:Table {idx: $idx_tab, env: $env}) "
                        "WHERE t.version = maximum "
                        "MERGE (c)-[r:IS_ATTRIBUTE_OF {weight: '1.0'}]->(t) "
                        "ON CREATE SET r.createdAt = timestamp(), r.updatetAt = timestamp() "
                        "ON MATCH SET r.updatetAt = timestamp() "
                        "RETURN 'relation ' + type(r) + ' created between ' + c.attribute + ' and ' + t.dataset", 
                        idx_tab=idx_tab, idx_col=idx_col, env=env)
        
        entire_result = []
        for record in result:
            entire_result.append(record[0])
            
        return entire_result

    @staticmethod
    def _create_column_column_relation(tx, idx_a, idx_b, weight, env):
        result = tx.run("MATCH (a:Column {idx: $idx_a, env: $env}), (b:Column {idx: $idx_b, env: $env}) "
                        "MERGE (a)-[r:IS_SIMILAR_TO {weight: $weight}]-(b) "
                        "ON CREATE SET r.createdAt = timestamp(), r.updatetAt = timestamp() "
                        "ON MATCH SET r.updatetAt = timestamp() "
                        "RETURN 'relation ' + type(r) + ' created between ' + a.attribute + ' and ' + b.attribute", 
                        idx_a=idx_a, idx_b=idx_b, env=env, weight=weight)
        
        entire_result = []
        for record in result:
            entire_result.append(record[0])
            
        return entire_result
    
    
    @staticmethod
    def _delete_self_relations(tx, env):
        result = tx.run("MATCH (a {env: $env})-[rel:IS_SIMILAR_TO]->(a) DELETE rel", env=env)
        
        entire_result = []
        for record in result:
            entire_result.append(record[0])
            
        return entire_result
   
    @staticmethod
    def _delete_all(tx, env):
        result = tx.run("MATCH (n {env: $env}) DETACH DELETE n", env=env)
        
        entire_result = []
        for record in result:
            entire_result.append(record[0])
            
        return entire_result
    
    @staticmethod
    def _fetch_related_tables_by_signature_to_2_grade(tx, dataset, weight, dastart, daend, env):
        q = '''
            MATCH (a:Table)-[r1:IS_SIMILAR_TO]-(b:Table) 
            WHERE a.dataset = $dataset AND a.env = $env AND b.env = $env AND r1.weight = $weight
                AND $dastart <= r1.updatetAt AND r1.updatetAt <= $daend
            OPTIONAL MATCH (b:Table)-[r2:IS_SIMILAR_TO]-(c:Table) 
            WHERE c.env = $env AND c <> a AND r2.weight = $weight 
                AND $dastart <= r2.updatetAt AND r2.updatetAt <= $daend
            RETURN a, b, c
        '''
        
        result = tx.run(dedent(q), env=env, dataset=dataset, weight=weight, dastart=dastart, daend=daend)

        entire_result = []
        for record in result:
            entire_result.append((record[0], record[1], record[2]))

        return entire_result
    
    @staticmethod
    def _fetch_related_tables_by_content_to_1_grade(tx, attribute, weight, dastart, daend, env):
        q = '''
            MATCH (a:Column)-[r1:IS_ATTRIBUTE_OF]-(b:Table)<-[r2:IS_ATTRIBUTE_OF]-(c:Column)-[r3:IS_SIMILAR_TO]-(d:Column)-[r4:IS_ATTRIBUTE_OF]->(e:Table)
            WHERE a.attribute = $attribute AND a.env=$env AND b.env=$env AND c.env=$env AND d.env=$env AND e.env=$env AND b <> e AND r3.weight=$weight
                AND $dastart <= r1.updatetAt AND r1.updatetAt <= $daend
                AND $dastart <= r2.updatetAt AND r2.updatetAt <= $daend
                AND $dastart <= r3.updatetAt AND r3.updatetAt <= $daend
                AND $dastart <= r4.updatetAt AND r4.updatetAt <= $daend
            RETURN b, c, d, e
        '''
        
        result = tx.run(dedent(q), env=env, attribute=attribute, weight=weight, dastart=dastart, daend=daend)

        entire_result = []
        for record in result:
            entire_result.append((record[0], record[1], record[2], record[3]))

        return entire_result


In [43]:
%%local

env = 'dev'
reset_on_restart = False
debug = False

cache = StrictRedis()
es = Elasticsearch()
graph = GraphStorage("bolt://localhost:7687", "neo4j", "password", env, debug)

pool = redis.ConnectionPool(host='localhost', port=6379, db=4)
version_cache = redis.Redis(connection_pool=pool)

index_content = {
    'dev': 'dev-havel-index-content',
    'test': 'test-havel-index-content',
    'prod': 'prod-havel-index-content'
}

index_signature = {
    'dev': 'dev-havel-index-signature',
    'test': 'test-havel-index-signature',
    'prod': 'prod-havel-index-signature'
}

es_res_max_size = '10000'

if (reset_on_restart):
    clear_cache_ns('db0')
    clear_cache_ns('db1')
    clear_cache_ns('db2')
    clear_cache_ns('db3')
    
    clear_es(env)
    create_es_schema_content(env)
    create_es_schema_signature(env)
    
    graph.delete_all()   


lsh = {
    '0.5':  MinHashLSH(
        threshold=0.5, num_perm=128, storage_config={
        'type': 'redis',
        'basename': bytearray(index_content[env], 'utf-8'),
        'redis': {'host': 'localhost', 'port': 6379, 'db': 0}
    }),
    '0.7': MinHashLSH(
        threshold=0.7, num_perm=128, storage_config={
        'type': 'redis',
        'basename': bytearray(index_content[env], 'utf-8'),
        'redis': {'host': 'localhost', 'port': 6379, 'db': 1}
    }),
    '0.9': MinHashLSH(
        threshold=0.9, num_perm=128, storage_config={
        'type': 'redis',
        'basename': bytearray(index_content[env], 'utf-8'),
        'redis': {'host': 'localhost', 'port': 6379, 'db': 2}
     })
}

lsh_sig = {
    '0.8': MinHashLSH(
        threshold=0.8, num_perm=128, storage_config={
        'type': 'redis',
        'basename': bytearray(index_content[env], 'utf-8'),
        'redis': {'host': 'localhost', 'port': 6379, 'db': 3}
     })
}


# Create Profiles
for source, schema, dataset, url, df in config_data: create_profiles_signature(source, schema, dataset, df, env, lsh_sig)
for source, schema, dataset, url, df in config_data: create_profiles_content(source, schema, dataset, df, env, lsh)

# Create Table nodes
nodes, edges = get_nodes_and_edges(env, index_signature[env], lsh_sig)
for hit in nodes: graph.insert_table_node(hit['_id'], hit['_source'])
for k,b,th in edges:
    for v in b: 
        if debug: print("key: %s - bucket: %s - threshold: %s"%(k, b, th))
        graph.add_table_table_relation(k,v,th)

# Create Column nodes
nodes, edges = get_nodes_and_edges(env, index_content[env], lsh)
for hit in nodes: graph.insert_column_node(hit['_id'], calc_idx_signature(hit['_source']), hit['_source'])
for k,b,th in edges:
    for v in b: 
        if debug: print("key: %s - bucket: %s - threshold: %s"%(k, b, th))
        graph.add_column_column_relation(k,v,th)

# Cleanup
graph.remove_self_relations()

graph.close()

b'{"hash": "778cf3734337f29c14b27816f0a7b7de", "version": 5}'
{'hash': '778cf3734337f29c14b27816f0a7b7de', 'version': 5}
adventureworks-for-postgres-person-address
b'{"hash": "d3ceff9c1ba2f60621c24fd72bd6629c", "version": 1}'
{'hash': 'd3ceff9c1ba2f60621c24fd72bd6629c', 'version': 1}
adventureworks-for-postgres-person-addresstype
b'{"hash": "e08d5d9a9c77d127c4f6ca62a693bce8", "version": 1}'
{'hash': 'e08d5d9a9c77d127c4f6ca62a693bce8', 'version': 1}
adventureworks-for-postgres-person-businessentity
b'{"hash": "18da5d35d8cc9638ffca9f5328607351", "version": 1}'
{'hash': '18da5d35d8cc9638ffca9f5328607351', 'version': 1}
adventureworks-for-postgres-person-businessentityaddress
b'{"hash": "50b8bf6d8581675def8ba6ef96837c9e", "version": 1}'
{'hash': '50b8bf6d8581675def8ba6ef96837c9e', 'version': 1}
adventureworks-for-postgres-person-businessentitycontact
b'{"hash": "45feb6c7d21b8d4cd11189a75f7df563", "version": 1}'
{'hash': '45feb6c7d21b8d4cd11189a75f7df563', 'version': 1}
adventureworks-for-p

In [57]:
%%local 

import time
from datetime import datetime

daend = int(datetime.timestamp(datetime.now())*1000)

a = datetime(2022, 11, 23, 12, 12, 12)
dastart = int(datetime.timestamp(a)*1000)

print(dastart)
print(daend)

graph = GraphStorage("bolt://localhost:7687", "neo4j", "password", env, False)
es = Elasticsearch()

# Eminhizer
res = search_dataset_for_topic(index_content[env], "Firestone Drive")
print(res)

# find related datasets

for r in res:
    related_nodes_by_signature_to_2_grade = graph.find_related_tables_by_signature_to_2_grade(r['dataset'], '0.8', dastart, daend)
    
    print(related_nodes_by_signature_to_2_grade)
    for node in related_nodes_by_signature_to_2_grade: 
        print(str(node[0]['idx']) + ' - ')
        print(str(node[1]['idx']) + ' - ')
        print(str(node[2]['idx']) + '\n')
        
for r in res:
    related_nodes_by_content_to_1_grad = graph.find_related_tables_by_content_to_1_grad(r['attribute'], '0.5', dastart, daend)

    for node in related_nodes_by_content_to_1_grad: 
        print(str(node[0]['dataset']) + ' version: '+ str(node[0]['version']) + ' -> (' + str(node[1]['attribute']) + ' - ' + str(node[2]['attribute']) + ') <- ' + str(node[3]['dataset']) + ' version: '+ str(node[3]['version']) + '\n')
    
    print('- - - - - - \n')
    print('- - - - - - \n')
    print('- - - - - - \n')

1669201932000
1669226950274
[{'id': 'adventureworks-for-postgres-person-address-addressline1', 'score': 12.170639, 'source': 'adventureworks-for-postgres', 'schema': 'person', 'dataset': 'address', 'attribute': 'addressline1'}, {'id': 'adventureworks-for-postgres-person-address_2-addressline1', 'score': 12.170639, 'source': 'adventureworks-for-postgres', 'schema': 'person', 'dataset': 'address_2', 'attribute': 'addressline1'}]
[(<Node element_id='4:ef40828f-517e-4346-9649-9f38ec1a4def:108' labels=frozenset({'Table'}) properties={'schema': 'person', 'createdAt': 1669224737572, 'source': 'adventureworks-for-postgres', 'env': 'dev', 'idx': 'adventureworks-for-postgres-person-address', 'dataset': 'address', 'version': 5, 'updatetAt': 1669226230913}>, <Node element_id='4:ef40828f-517e-4346-9649-9f38ec1a4def:5' labels=frozenset({'Table'}) properties={'schema': 'person', 'createdAt': 1669224552856, 'source': 'adventureworks-for-postgres', 'env': 'dev', 'idx': 'adventureworks-for-postgres-pers

In [ ]:
%%local 

graph = GraphStorage("bolt://localhost:7687", "neo4j", "password", env, False)
es = Elasticsearch()

res = search_dataset_for_topic(index_content[env], "Eminhizer")

# find related datasets

related_datasets = {
    'related_nodes_by_signature_to_2_grade': [],
    'related_nodes_by_content_to_1_grad': []
}

for r in res:
    related_datasets['related_nodes_by_signature_to_2_grade'].append(graph.find_related_tables_by_signature_to_2_grade(r['dataset'], '0.8'))

for r in res:
    related_datasets['related_nodes_by_content_to_1_grad'].append(graph.find_related_tables_by_content_to_1_grad(r['id'], '0.5'))
    

for node in related_datasets['related_nodes_by_signature_to_2_grade']:
    print(node)
    if (node):
        print(str(node[0]['name']) + ' - ')
        print(str(node[1]['name']) + ' - ')
        print(str(node[2]['name']) + '\n')\

for node in related_datasets['related_nodes_by_content_to_1_grad']:
    print(node)
    if node:
        a = node[0]
        print(str(a['name']) + ' -> (' + str(node[1]['name']) + ' - ' + str(node[2]['name']) + ') <- ' + str(node[3]['name']) + '\n')


In [78]:
%%local 

arr = np.array([ 42114592,  127266168,  215678469,   17266034,   34121180,  942441018,
  227629861,  772424727,  258055101,  289724735,  117062477,  555008702,
   29604049,  840321879,  378698042, 1209184744,  243903852,   64306427,
  418573613,   50885453,  443551390,  238376316,  629454598,  646390910,
  258707738,  287206176,   86790830,  230629753,  353312205,   72417695,
  113209494,  247807691,  494356438,  598711389,  163905412,   92705481,
  167455200,  120333262,  690658470,  503937613,   13774441,  138757320,
  429215096,  166965839,  604024314,  151611451,  482383227,   80185077,
  132702729,  637863885,  809374076,   15601261,  231429825,  508796949,
   93190304,  246310319,  252884464,   76719615,   27566880,  600163519,
    9727857,   11786542,  156050497,  303860790,  228751560,  912519144,
    9156249,  369895830,  727826405,  416426394, 1081480011,   66120612,
  135376155,  210143265, 1031356013,   90284798, 1016601632,   13573364,
   68970483,  161433822,  542646600,   96345609,  685539749,   84541516,
   86327512,  187842457,  132296212,  857191589,   20737788,  590826688,
  444073826,  192175731,  212992765,  114574843,  171782812,   70248969,
  202706043,  702713690,  338155350,  342468131,  542793283,  261167155,
  378396244,  211528111,  231333263,  103754312,  129317962,   55448546,
 1041271092,  594880347,  931295081,  225781925,   83509603,  776091994,
  298124272,  467369673,    8909661,   60688953,  115820188,   54821235,
  168100490,   33183095,  712469771,  323850472,  414195156,  550484318,
  388634331,   42270916], dtype=np.int64)

mt = MinHash(num_perm=128)
mt.hashvalues = arr
mt.seed = 1

lsh['0.7'].query(mt)


[]